In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timezone
import calendar
import os
from google.colab import drive

# -------------------------------
# 1️⃣ Google Drive Mount
# -------------------------------
drive.mount('/content/drive')

# Drive میں فولڈر جہاں CSV save ہوگی
DRIVE_PATH = "/content/drive/MyDrive/UrduNovelBank"
os.makedirs(DRIVE_PATH, exist_ok=True)

# -------------------------------
# 2️⃣ Blog & RSS Settings
# -------------------------------
BLOG_URL = "https://www.urdunovelbanks.com/"
RSS_URL = BLOG_URL + "feeds/posts/default?alt=rss"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0 Safari/537.36"
}

posts_data = []

# -------------------------------
# 3️⃣ RSS Fetch
# -------------------------------
try:
    rss = requests.get(RSS_URL, headers=headers, timeout=15)
    rss.raise_for_status()
except Exception as e:
    print("RSS فیڈ لوڈ کرنے میں مسئلہ:", e)
    exit()

soup = BeautifulSoup(rss.content, "xml")
print("Total RSS items in feed:", len(soup.find_all("item")))

# موجودہ UTC تاریخ
today = datetime.now(timezone.utc)
current_year = today.year
current_month = today.month

print(f"\nموجودہ مہینے کی پوسٹس نکال رہے ہیں: {calendar.month_name[current_month]} {current_year}")

# -------------------------------
# 4️⃣ Extract Posts
# -------------------------------
for item in soup.find_all("item"):
    title = item.title.text.strip()
    post_url = item.link.text
    pub_date_str = item.pubDate.text

    try:
        pub_date = datetime.strptime(pub_date_str, "%a, %d %b %Y %H:%M:%S %z")
        pub_date_only = pub_date.date()
    except Exception as e:
        print(f"تاریخ پارٹس کرنے میں مسئلہ ({title}):", e)
        continue

    # صرف موجودہ مہینے کی پوسٹس
    if pub_date_only.year == current_year and pub_date_only.month == current_month:
        try:
            post_html = requests.get(post_url, headers=headers, timeout=15).text
            post_soup = BeautifulSoup(post_html, "lxml")

            # Post content
            content_div = post_soup.find("div", class_=re.compile("post-body", re.I))
            content_html = str(content_div) if content_div else ""

            # Image
            img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html, re.I)
            image_url = img_match.group(1) if img_match else ""

            # Labels extraction (fixed for current site)
            labels = []
            for a in post_soup.select('.label-container .label-link'):
                labels.append(a.get_text(strip=True))
            labels = list(set(labels))  # Remove duplicates

        except Exception as e:
            print(f"Post fetch error ({title}):", e)
            content_html = image_url = ""
            labels = []

        posts_data.append({
            "Date": pub_date_only.strftime("%Y-%m-%d"),
            "Title": title,
            "Image URL": image_url,
            "Description (HTML)": content_html,
            "Labels": ", ".join(labels),
            "Post URL": post_url,
        })

# -------------------------------
# 5️⃣ Save to CSV (Google Drive)
# -------------------------------
df = pd.DataFrame(posts_data)

if len(df) > 0:
    print("\nملنے والی پوسٹس:")
    print(df[['Date', 'Title', 'Post URL', 'Labels']])

    filename = f"{DRIVE_PATH}/urdunovelbanks_{current_year}-{current_month:02d}_posts.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\nڈیٹا '{filename}' فائل میں سیو ہو گیا۔ گوگل شیٹس میں اپ لوڈ کر لیں۔")
else:
    print("\nموجودہ مہینے میں کوئی پوسٹ نہیں ملی۔")

print("\nسمری: موجودہ مہینے میں کل پوسٹس کی تعداد:", len(df))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total RSS items in feed: 25

موجودہ مہینے کی پوسٹس نکال رہے ہیں: December 2025

ملنے والی پوسٹس:
          Date                                              Title  \
0   2025-12-20  Video: Yar yaroon sy hona juda novel by Zainab...   
1   2025-12-20  Video: Mala novel by Nimra Ahmed | Mahir faree...   
2   2025-12-18  Video: Some Incomplete Characters in Complete ...   
3   2025-12-18  Magar ik sitara mehrban By Aalia Bukhari – Rea...   
4   2025-12-18  Aik thi baila novel by Alia Bukhari – Read Onl...   
5   2025-12-18  Aik ansoo aik sitara by Alia Bukhari – Read On...   
6   2025-12-17  Abhi Manzar Badalna Hai by Aliya Bukhari – Rea...   
7   2025-12-17  Mahe Tamam by parveen shakir – Read Online & D...   
8   2025-12-17  Kulliyat e parveen shakir – Read Online & Down...   
9   2025-12-15  Shehar e Ashob By Aalia Bukhari – Read Online ...   
10  2025-12-15 

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timezone
import calendar
import os
from google.colab import drive

# -------------------------------
# 1️⃣ Google Drive Mount
# -------------------------------
drive.mount('/content/drive')

# Drive میں فولڈر جہاں CSV save ہوگی
DRIVE_PATH = "/content/drive/MyDrive/UrduNovelBank"
os.makedirs(DRIVE_PATH, exist_ok=True)

# -------------------------------
# 2️⃣ Blog & RSS Settings
# -------------------------------
BLOG_URL = "https://www.urdunovelbanks.com/"
RSS_URL = BLOG_URL + "feeds/posts/default?alt=rss"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0 Safari/537.36"
}

posts_data = []

# -------------------------------
# 3️⃣ RSS Fetch
# -------------------------------
try:
    rss = requests.get(RSS_URL, headers=headers, timeout=15)
    rss.raise_for_status()
except Exception as e:
    print("RSS فیڈ لوڈ کرنے میں مسئلہ:", e)
    exit()

soup = BeautifulSoup(rss.content, "xml")
print("Total RSS items in feed:", len(soup.find_all("item")))

# موجودہ UTC تاریخ
today = datetime.now(timezone.utc)
current_year = today.year
current_month = today.month

print(f"\nموجودہ مہینے کی پوسٹس نکال رہے ہیں: {calendar.month_name[current_month]} {current_year}")

# -------------------------------
# 4️⃣ Extract Posts
# -------------------------------
for item in soup.find_all("item"):
    title = item.title.text.strip()
    post_url = item.link.text
    pub_date_str = item.pubDate.text

    try:
        pub_date = datetime.strptime(pub_date_str, "%a, %d %b %Y %H:%M:%S %z")
        pub_date_only = pub_date.date()
    except Exception as e:
        print(f"تاریخ پارٹس کرنے میں مسئلہ ({title}):", e)
        continue

    # صرف موجودہ مہینے کی پوسٹس
    if pub_date_only.year == current_year and pub_date_only.month == current_month:
        try:
            post_html = requests.get(post_url, headers=headers, timeout=15).text
            post_soup = BeautifulSoup(post_html, "lxml")

            # Post content
            content_div = post_soup.find("div", class_=re.compile("post-body", re.I))
            content_html = str(content_div) if content_div else ""

            # Image
            img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html, re.I)
            image_url = img_match.group(1) if img_match else ""

            # Labels extraction (improved for Blogger)
            labels = []
            # Method 1: rel="tag"
            for a in post_soup.select('a[rel="tag"]'):
                labels.append(a.get_text(strip=True))
            # Method 2: class="post-labels" (Blogger common structure)
            if not labels:
                for a in post_soup.select('.post-labels a'):
                    labels.append(a.get_text(strip=True))
            labels = list(set(labels))  # Remove duplicates

        except Exception as e:
            print(f"Post fetch error ({title}):", e)
            content_html = image_url = ""
            labels = []

        posts_data.append({
            "Date": pub_date_only.strftime("%Y-%m-%d"),
            "Title": title,
            "Image URL": image_url,
            "Description (HTML)": content_html,
            "Labels": ", ".join(labels),
            "Post URL": post_url,
        })

# -------------------------------
# 5️⃣ Save to CSV (Google Drive)
# -------------------------------
df = pd.DataFrame(posts_data)

if len(df) > 0:
    print("\nملنے والی پوسٹس:")
    print(df[['Date', 'Title', 'Post URL', 'Labels']])

    filename = f"{DRIVE_PATH}/urdunovelbanks_{current_year}-{current_month:02d}_posts.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\nڈیٹا '{filename}' فائل میں سیو ہو گیا۔ گوگل شیٹس میں اپ لوڈ کر لیں۔")
else:
    print("\nموجودہ مہینے میں کوئی پوسٹ نہیں ملی۔")

print("\nسمری: موجودہ مہینے میں کل پوسٹس کی تعداد:", len(df))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total RSS items in feed: 25

موجودہ مہینے کی پوسٹس نکال رہے ہیں: December 2025

ملنے والی پوسٹس:
          Date                                              Title  \
0   2025-12-20  Video: Yar yaroon sy hona juda novel by Zainab...   
1   2025-12-20  Video: Mala novel by Nimra Ahmed | Mahir faree...   
2   2025-12-18  Video: Some Incomplete Characters in Complete ...   
3   2025-12-18  Magar ik sitara mehrban By Aalia Bukhari – Rea...   
4   2025-12-18  Aik thi baila novel by Alia Bukhari – Read Onl...   
5   2025-12-18  Aik ansoo aik sitara by Alia Bukhari – Read On...   
6   2025-12-17  Abhi Manzar Badalna Hai by Aliya Bukhari – Rea...   
7   2025-12-17  Mahe Tamam by parveen shakir – Read Online & D...   
8   2025-12-17  Kulliyat e parveen shakir – Read Online & Down...   
9   2025-12-15  Shehar e Ashob By Aalia Bukhari – Read Online ...   
10  2025-12-15 

In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime, timezone
import calendar
import os
from google.colab import drive

# -------------------------------
# 1️⃣ Google Drive Mount
# -------------------------------
drive.mount('/content/drive')

# Drive میں فولڈر جہاں CSV save ہوگی
DRIVE_PATH = "/content/drive/MyDrive/UrduNovelBank"
os.makedirs(DRIVE_PATH, exist_ok=True)

# -------------------------------
# 2️⃣ Blog & RSS Settings
# -------------------------------
BLOG_URL = "https://www.urdunovelbanks.com/"
RSS_URL = BLOG_URL + "feeds/posts/default?alt=rss"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0 Safari/537.36"
}

posts_data = []

# -------------------------------
# 3️⃣ RSS Fetch
# -------------------------------
try:
    rss = requests.get(RSS_URL, headers=headers, timeout=15)
    rss.raise_for_status()
except Exception as e:
    print("RSS فیڈ لوڈ کرنے میں مسئلہ:", e)
    exit()

soup = BeautifulSoup(rss.content, "xml")
print("Total RSS items in feed:", len(soup.find_all("item")))

# موجودہ UTC تاریخ
today = datetime.now(timezone.utc)
current_year = today.year
current_month = today.month

print(f"\nموجودہ مہینے کی پوسٹس نکال رہے ہیں: {calendar.month_name[current_month]} {current_year}")

# -------------------------------
# 4️⃣ Extract Posts
# -------------------------------
for item in soup.find_all("item"):
    title = item.title.text.strip()
    post_url = item.link.text
    pub_date_str = item.pubDate.text

    try:
        pub_date = datetime.strptime(pub_date_str, "%a, %d %b %Y %H:%M:%S %z")
        pub_date_only = pub_date.date()
    except Exception as e:
        print(f"تاریخ پارٹس کرنے میں مسئلہ ({title}):", e)
        continue

    # صرف موجودہ مہینے کی پوسٹس
    if pub_date_only.year == current_year and pub_date_only.month == current_month:
        # اضافی ڈیٹا (امیج، لیبلز وغیرہ)
        try:
            post_html = requests.get(post_url, headers=headers, timeout=15).text
            post_soup = BeautifulSoup(post_html, "lxml")

            content_div = post_soup.find("div", class_=re.compile("post-body", re.I))
            content_html = str(content_div) if content_div else ""

            img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html, re.I)
            image_url = img_match.group(1) if img_match else ""

            labels = [a.get_text(strip=True) for a in post_soup.select('a[rel="tag"]')]
        except Exception:
            content_html = image_url = ""
            labels = []

        posts_data.append({
            "Date": pub_date_only.strftime("%Y-%m-%d"),
            "Title": title,
            "Image URL": image_url,
            "Description (HTML)": content_html,
            "Labels": ", ".join(labels),
            "Post URL": post_url,
        })

# -------------------------------
# 5️⃣ Save to CSV (Google Drive)
# -------------------------------
df = pd.DataFrame(posts_data)

if len(df) > 0:
    print("\nملنے والی پوسٹس:")
    print(df[['Date', 'Title', 'Post URL']])

    filename = f"{DRIVE_PATH}/urdunovelbanks_{current_year}-{current_month:02d}_posts.csv"
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"\nڈیٹا '{filename}' فائل میں سیو ہو گیا۔ گوگل شیٹس میں اپ لوڈ کر لیں۔")
else:
    print("\nموجودہ مہینے میں کوئی پوسٹ نہیں ملی۔")

print("\nسمری: موجودہ مہینے میں کل پوسٹس کی تعداد:", len(df))


Mounted at /content/drive
Total RSS items in feed: 25

موجودہ مہینے کی پوسٹس نکال رہے ہیں: December 2025

ملنے والی پوسٹس:
          Date                                              Title  \
0   2025-12-20  Video: Yar yaroon sy hona juda novel by Zainab...   
1   2025-12-20  Video: Mala novel by Nimra Ahmed | Mahir faree...   
2   2025-12-18  Video: Some Incomplete Characters in Complete ...   
3   2025-12-18  Magar ik sitara mehrban By Aalia Bukhari – Rea...   
4   2025-12-18  Aik thi baila novel by Alia Bukhari – Read Onl...   
5   2025-12-18  Aik ansoo aik sitara by Alia Bukhari – Read On...   
6   2025-12-17  Abhi Manzar Badalna Hai by Aliya Bukhari – Rea...   
7   2025-12-17  Mahe Tamam by parveen shakir – Read Online & D...   
8   2025-12-17  Kulliyat e parveen shakir – Read Online & Down...   
9   2025-12-15  Shehar e Ashob By Aalia Bukhari – Read Online ...   
10  2025-12-15  Abhi kuch zakham silne baqi hen by Aliya Bukha...   
11  2025-12-15  Khawab Siraaye By Aalia Bukhari –

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Added 'max-results=500' to get more than the default 25 posts
BLOG_URL = "https://www.urdunovelbanks.com/"
RSS_URL = BLOG_URL + "/feeds/posts/default?alt=rss&max-results=500"

posts_data = []

print(f"Fetching RSS Feed: {RSS_URL} ...")
rss = requests.get(RSS_URL)
soup = BeautifulSoup(rss.content, "xml")

items = soup.find_all("item")
print(f"Total RSS items found: {len(items)}")

for item in items:
    # 1. Extract Title
    title = item.title.text

    # 2. Extract Post URL
    post_url = item.link.text

    # 3. Extract Content (Description contains the full HTML in Blogger RSS)
    content_html = item.description.text if item.description else ""

    # 4. Extract Image
    # First check for media:thumbnail provided by Blogger
    thumbnail = item.find("media:thumbnail")
    if thumbnail:
        image_url = thumbnail["url"]
        # Optional: Convert thumbnail 's72-c' size to full size if needed
        # image_url = image_url.replace("/s72-c/", "/s1600/")
    else:
        # Fallback: Find first image in content
        img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html)
        image_url = img_match.group(1) if img_match else ""

    # 5. Extract Labels (Categories)
    # In RSS, labels are stored in <category> tags
    categories = item.find_all("category")
    labels = [cat.text for cat in categories]

    posts_data.append({
        "Title": title,
        "Image URL": image_url,
        "Description (HTML)": content_html, # Contains the full post body
        "Labels": ", ".join(labels),
        "Post URL": post_url,
    })

df = pd.DataFrame(posts_data)

# Print first few rows to verify
print(df.head())

# Optional: Save to CSV
# df.to_csv("urdu_novels_data.csv", index=False)

Fetching RSS Feed: https://www.urdunovelbanks.com//feeds/posts/default?alt=rss&max-results=500 ...
Total RSS items found: 150
                                               Title  \
0  Video: Yar yaroon sy hona juda novel by Zainab...   
1  Video: Mala novel by Nimra Ahmed | Mahir faree...   
2  Video: Some Incomplete Characters in Complete ...   
3  Magar ik sitara mehrban By Aalia Bukhari – Rea...   
4  Aik thi baila novel by Alia Bukhari – Read Onl...   

                                           Image URL  \
0  https://blogger.googleusercontent.com/img/b/R2...   
1  https://blogger.googleusercontent.com/img/b/R2...   
2  https://blogger.googleusercontent.com/img/b/R2...   
3  https://blogger.googleusercontent.com/img/b/R2...   
4  https://blogger.googleusercontent.com/img/b/R2...   

                                  Description (HTML)  \
0  <div class="separator" style="clear: both; tex...   
1  <div class="separator" style="clear: both; tex...   
2  <div class="separator" style=

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

BLOG_URL = "https://www.urdunovelbanks.com/"
RSS_URL = BLOG_URL + "/feeds/posts/default?alt=rss"

posts_data = []

rss = requests.get(RSS_URL)
soup = BeautifulSoup(rss.content, "xml")

print("Total RSS items:", len(soup.find_all("item")))

for item in soup.find_all("item"):

    post_url = item.link.text
    post_html = requests.get(post_url).text
    post_soup = BeautifulSoup(post_html, "lxml")

    title = item.title.text

    content_div = post_soup.find("div", class_=re.compile("post-body"))
    content_html = str(content_div) if content_div else ""

    img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html)
    image_url = img_match.group(1) if img_match else ""

    labels = [a.get_text(strip=True) for a in post_soup.select('a[rel="tag"]')]

    posts_data.append({
        "Title": title,
        "Image URL": image_url,
        "Description (HTML)": content_html,
        "Labels": ", ".join(labels),
        "Post URL": post_url,
    })

df = pd.DataFrame(posts_data)
df.head()


Total RSS items: 25


,Title,Image URL,Description (HTML),Labels,Post URL
0,Video: Yar yaroon sy hona juda novel by Zainab...,https://blogger.googleusercontent.com/img/b/R2...,"<div class=""post-body entry-content"" id=""postB...",,https://www.urdunovelbanks.com/2025/12/video-y...
1,Video: Mala novel by Nimra Ahmed | Mahir faree...,https://blogger.googleusercontent.com/img/b/R2...,"<div class=""post-body entry-content"" id=""postB...",,https://www.urdunovelbanks.com/2025/12/video-m...
2,Video: Some Incomplete Characters in Complete ...,https://blogger.googleusercontent.com/img/b/R2...,"<div class=""post-body entry-content"" id=""postB...",,https://www.urdunovelbanks.com/2025/12/video-s...
3,Magar ik sitara mehrban By Aalia Bukhari – Rea...,https://blogger.googleusercontent.com/img/b/R2...,"<div class=""post-body entry-content"" id=""postB...",,https://www.urdunovelbanks.com/2025/12/magar-i...
4,Aik thi baila novel by Alia Bukhari – Read Onl...,https://blogger.googleusercontent.com/img/b/R2...,"<div class=""post-body entry-content"" id=""postB...",,https://www.urdunovelbanks.com/2025/12/aik-thi...


In [5]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1G58dzl6K7em19YIJ1iQ_f7n-6ghsMDWKd3ZWR-vKkO4/edit#gid=0


In [3]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1u7j0451h1nkoGIwoyIuq8lDrIaNzNDQ5urVSB6mh17A/edit#gid=0


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
import re

BLOG_URL = "https://www.urdunovelbanks.com/"   # 👈 apna blog
ARCHIVE_URL = BLOG_URL + "/feeds/posts/default?alt=rss"

# 🔹 Last month range
today = datetime.today()
first_day_last_month = (today.replace(day=1) - timedelta(days=1)).replace(day=1)
last_day_last_month = today.replace(day=1) - timedelta(days=1)

posts_data = []

rss = requests.get(ARCHIVE_URL)
soup = BeautifulSoup(rss.content, "xml")

for item in soup.find_all("item"):
    pub_date = datetime.strptime(item.pubDate.text[:16], "%a, %d %b %Y")

    if not (first_day_last_month <= pub_date <= last_day_last_month):
        continue

    post_url = item.link.text
    post_html = requests.get(post_url).text
    post_soup = BeautifulSoup(post_html, "lxml")

    # 🔹 Title
    title = post_soup.find("h1")
    title = title.get_text(strip=True) if title else item.title.text

    # 🔹 Full HTML Content
    content_div = post_soup.find("div", class_=re.compile("post-body"))
    content_html = str(content_div) if content_div else ""

    # 🔹 First Image
    img_match = re.search(r'<img[^>]+src="([^">]+)"', content_html)
    image_url = img_match.group(1) if img_match else ""

    # 🔹 Labels
    labels = [a.get_text(strip=True) for a in post_soup.select('a[rel="tag"]')]

    posts_data.append({
        "Title": title,
        "Image URL": image_url,
        "Description (HTML)": content_html,
        "Labels": ", ".join(labels),
        "Post URL": post_url,
        "Published Date": pub_date.strftime("%Y-%m-%d")
    })

# 🔹 Save Excel
df = pd.DataFrame(posts_data)
file_name = f"Blogger_Backup_{first_day_last_month.strftime('%Y_%m')}.xlsx"
df.to_excel(file_name, index=False)

df.head(), file_name


(Empty DataFrame
 Columns: []
 Index: [],
 'Blogger_Backup_2025_11.xlsx')